- Here we explore the use of `vtki`to create 3D objects we then display using `ipyvolume`.
- In the future we might want to use `itkwidgets` to display 3d objects. See https://github.com/InsightSoftwareConsortium/itk-jupyter-widgets/issues/112


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../../")

import anamic

import numpy as np
import matplotlib.pyplot as plt
import ipyvolume as ipv
import vtki

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
n_pf = 11
mt_length_nm = 100  # nm
taper_length_nm = 0  # nm

# Create a MicrotubuleSimulator object.
dimers = anamic.simulator.dimers_builder(n_pf, mt_length_nm, taper_length_nm)

# Set parameters for the image generation.
parameters = {}
parameters['labeling_ratio'] = 0.1  # from 0 to 1

parameters['pixel_size'] = 110  # nm/pixel
parameters['x_offset'] = 1500  # nm
parameters['y_offset'] = 1500  # nm

parameters['psf_size'] = 135  # nm

parameters['signal_mean'] = 700
parameters['signal_std'] = 100
parameters['bg_mean'] = 500
parameters['bg_std'] = 24
parameters['noise_factor'] = 1

parameters['snr_line_width'] = 3  # pixel

ms = anamic.simulator.MicrotubuleSimulator(dimers)
ms.parameters.update(parameters)

# Build the geometry.
ms.build_positions(apply_random_z_rotation=True, show_progress=True)
ms.label()
ms.project()
ms.random_rotation_projected()

100%|██████████| 11/11 [00:00<00:00, 3865.07it/s]


In [4]:
model_path = Path('/home/hadim/Documents/Code/Postdoc/tubulin-dimer.stl')
dimer = vtki.PolyData(str(model_path))

# Move dimer to center of mass
mass_center = dimer.points.mean(axis=0)
dimer.translate(mass_center * -1)

In [ ]:
x = dimer.points[:, 0]
y = dimer.points[:, 1]
z = dimer.points[:, 2]

# The faces returned by `PolyData` can be different
# than triangle, so we assume `vtki.Sphere.faces` returns
# only triangles.
triangles = np.array([dimer.faces[1::4], dimer.faces[2::4], dimer.faces[3::4]]).T

ipv.figure()
ipv.plot_trisurf(x, y, z, triangles=triangles, color='orange')
ipv.xyzlim(-50, 50)
ipv.show()